In [3]:
from bs4 import BeautifulSoup

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import GlobalAveragePooling2D as GAP2D
from tensorflow.keras.layers import Dropout, Dense, Conv2D
from tensorflow.keras.optimizers import Adam

import numpy as np

import os
from PIL import Image
from IPython.display import clear_output\

from matplotlib import pyplot as plt

In [4]:
bbox_cords = ['xmin', 'ymin', 'xmax', 'ymax']
classnames={'cucumber':0, 'eggplant':1, 'mushroom':2}
imgs = []
y = []
bbox = []

for ind, files in enumerate(os.listdir('dataset/')):
    if files.find('.xml')!=-1:
        with open(f'dataset/{files}', 'r') as f:
            data = f.read()
        Bs_data = BeautifulSoup(data, "xml")
        img_name = Bs_data.find('filename').string
        q = Image.open('dataset/' + img_name)
        
        imgs.append(np.array(q))
        y.append(classnames[img_name[:img_name.find('_')]])
        
        bbox.append(np.array([int(Bs_data.find(i).string) for i in bbox_cords]))

imgs=np.array(imgs, dtype='float64')
y=tf.one_hot(np.array(y), 3)
bbox=np.array(bbox, dtype='float64')

imgs/=127.5
imgs-=1
bbox/=113.5
bbox-=1

In [5]:
imgs.shape, y.shape, bbox.shape

((186, 227, 227, 3), TensorShape([186, 3]), (186, 4))

In [8]:
def build_model():
    inp = tf.keras.Input(shape=(227, 227, 3))
    x = Conv2D(50,
               kernel_size=3,
               strides=1,
               activation='relu',
               name='conv1')(inp)
    x = Conv2D(50,
               kernel_size=3,
               strides=1,
               activation='relu',
               name='conv2')(x)
    x = GAP2D(name='GAP2D')(x)
    
    c = Dense(100, activation='relu', name='dense_c')(x)
    c = Dense(3, activation='sigmoid', name='classifier')(c) # classification layer
    
    b = Dense(100, activation='relu', name='dense_b')(x) # bbox layer
    b = Dense(4, name='detector')(b)
    
    model1 = tf.keras.Model(inp, (c, b))
    model1.compile(optimizer=Adam(),
                  loss={'classifier':'categorical_crossentropy', 'detector':'mse'},
                  metrics={'classifier':'accuracy', 'detector':'mse'})
    return model1

In [9]:
model = build_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 227, 227, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 225, 225, 50  1400        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2 (Conv2D)                 (None, 223, 223, 50  22550       ['conv1[0][0]']                  
                                )                                                           

In [10]:
model.fit(imgs, [y, bbox],
          epochs=20,
          batch_size=32,
          shuffle=True,
          verbose=1)

Epoch 1/20
6/6 [==============================] - 6s 916ms/step - loss: 1.4606 - classifier_loss: 1.1005 - detector_loss: 0.3600 - classifier_accuracy: 0.2957 - detector_mse: 0.3600
Epoch 2/20
6/6 [==============================] - 6s 947ms/step - loss: 1.2036 - classifier_loss: 1.0925 - detector_loss: 0.1111 - classifier_accuracy: 0.3602 - detector_mse: 0.1111
Epoch 3/20
6/6 [==============================] - 6s 911ms/step - loss: 1.1402 - classifier_loss: 1.0637 - detector_loss: 0.0764 - classifier_accuracy: 0.4839 - detector_mse: 0.0764
Epoch 4/20
6/6 [==============================] - 5s 917ms/step - loss: 1.0942 - classifier_loss: 1.0361 - detector_loss: 0.0581 - classifier_accuracy: 0.5054 - detector_mse: 0.0581
Epoch 5/20
6/6 [==============================] - 6s 933ms/step - loss: 1.0650 - classifier_loss: 1.0084 - detector_loss: 0.0565 - classifier_accuracy: 0.5645 - detector_mse: 0.0565
Epoch 6/20
6/6 [==============================] - 6s 921ms/step - loss: 1.0245 - classifie